In [1]:
import pandas as pd
from IPython.display import Markdown

from tulip.core.collection import TulipCollection
from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.haver import HaverClient as hc
from tulip.plots import plot_lines
from tulip.data.gs import GSClient as gs

import pycountry

from tulip.analysis.country_related.analytics import summarize_gs_eco_fct

ISO_2 = "DE"
BBERG_ISO_2 = ISO_2
pyctry = pycountry.countries.get(alpha_2=ISO_2)
ISO_3 = pyctry.alpha_3

Haver path setting remains unchanged.



## Germany

In [2]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:21_

### Activity Indicators
#### Economic Forecasts (Brokers)

In [3]:
gs_eco_fct = gs.get_eco_forecast(geographyId=ISO_2)
gs_summary = summarize_gs_eco_fct(gs_eco_fct)
gs_summary = gs_summary[~gs_summary.index.str.contains("ngdp")].to_frame().T
gs_summary.style.set_caption(f"Goldman {pyctry.name} Economic Forecasts").format(
    precision=2
)

metric,core_cpi,cpi_avg,current_account,rgdp_qoq,rgdp_yoy,output_gap
forecastValue,2.64,2.27,nan,1.31,0.31,-1.96


#### Current Activity Indicator (Goldman)

In [4]:
cai_series_soft_vs_hard = gs.get_CAI_series(
    geographyId=ISO_2,
    metricName=[
        "CAI_HEADLINE",
        "CAI_CONTRIBUTION_TYPE_HARD",
        "CAI_CONTRIBUTION_TYPE_SOFT",
    ],
    startDate="1980-01-01",
)
cai_series_soft_vs_hard = cai_series_soft_vs_hard.set_index("metricName", append=True)[
    "metricValue"
].unstack("metricName")
cai_series_soft_vs_hard.columns = ["Hard", "Soft", "Headline"]
cai_plot = plot_lines(
    cai_series_soft_vs_hard,
    show_0=True,
    title=f"<b>{pyctry.name} Current Activity Indicator</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
    years_limit=4,
)
cai_plot

In [5]:
try:
    nowcasts = bb.create_collection(["BENWEAGC Index", "BENWDEGC Index"])
    nowcasts.dashboard.table()
except Exception as e:
    print(f"There was an issue with downloading the nowcasts from Blooomberg")

In [6]:
try:
    nowcasts.dashboard.plots(show_0=True)
except:
    pass

- European Commission [Nowcasts](https://inflation-nowcast-socialplatform.streamlit.app/)
- ifoCast [Nowcast](https://inflation-nowcast-socialplatform.streamlit.app/)

#### High Frequency Stuff

<iframe src="https://www.dashboard-deutschland.de/iframe/tile_1667211885741?mtm_campaign=dd-embedding?darkmode=undefined" width="100%" height="600px" frameborder="0"></iframe>

- Development of the German labour market based on the LinkedIn [Hiring Rate](https://www.dashboard-deutschland.de/indicator/tile_1673880739519)

#### Retail Sales

In [7]:
retail_sales = hc.get_series(" yryr%(DESTU47@GERMANY)")
retail_sales.info.title = "Retail Sales MoM"
retail_sales.plot(years_limit=2, tick_suffix="%", show_0=True)

#### PMIs

In [8]:
pmi = {
    "MPMIDESA Index": "HCOB Germany Services PMI",
    "MPMIDECA Index": "HCOB Germany Composite PMII",
}

pmi_collection = bb.create_collection(list(pmi.keys()))
for i, k in enumerate(pmi.values()):
    pmi_collection[i].info.title = k

pmi_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Quote Units,Updated
HCOB Germany Services PMI,54.6,2025-10-31,51.5,3.1,1.5,5.6,3.0,-,2025-11-12 19:21:00
HCOB Germany Composite PMII,53.9,2025-10-31,52.0,1.9,1.1,3.8,5.3,-,2025-11-12 19:21:00


In [9]:
pmi_collection.dashboard.plots(hlines=50)

#### Industrial Production

In [10]:
indprod = {
    "GRIPIMOM Index": "Industrial Production SA MoM",
    "GRIORTMM Index": "Factory Orders MoM",
}

indprod_collection = bb.create_collection(list(indprod.keys()))
for i, k in enumerate(indprod.values()):
    indprod_collection[i].info.title = k

indprod_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Quote Units,Updated
Industrial Production SA MoM,1.3,2025-09-30,-3.7,5.0,1.8,2.3,2.7,-,2025-11-12 19:21:00
Factory Orders MoM,1.1,2025-09-30,-0.40,1.5,0.28,1.1,-5.9,-,2025-11-12 19:21:00


In [11]:
indprod_collection.dashboard.plots(years_limit=3, mma=12, tick_suffix="%", show_0=True)

#### Growth Stats
##### Real GDP Growth

In [12]:
real_gdp_yoy = hc.get_series("DEWNGDPW@GERMANY")
real_gdp_yoy.plot(show_0=True, tick_suffix="%", source="Haver")

##### GDP Detail

In [13]:
growth_stats = {
    "Real GDP": "yryr%(DESNGDPQ@GERMANY )",  # Canada: Real Gross Domestic Product at Market Prices
    "Fixed Investment": "yryr%(DESNFQ@GERMANY)",  # Canada: GDP: Gross Fixed Capital Formation
    "Govt Consumption": "yryr%(DESNCPQ@GERMANY)",  # Canada: GDP: Investment in Inventories
    "Private Consumption": "yryr%(DESNCVQ@GERMANY )",  # Canada: GDP: Final Consumption Expenditure
    "Exports": "yryr%(DESNXQ@GERMANY)",  # Canada: GDP: Exports of Goods and Services
    "Imports": "yryr%(DESNMQ@GERMANY)",  # Canada: GDP: Imports of Goods and Services
}

growth_stats_collection = hc.create_collection(list(growth_stats.values()))
growth_stats_collection["yryr%(DESNMQ@GERMANY)"].good_is = -1  # Flip what Good is
growth_stats_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Germany: GDP: Total Imports (SWDA, 2020=100)",3.8,2025-06-30,3.6,0.16,0.04,3.1,4.8,2025-11-12 19:21:00
"Germany: GDP: Private Consumption (SWDA, 2020=100)",1.3,2025-06-30,1.3,-0.07,-0.03,0.67,1.5,2025-11-12 19:21:00
"Germany: GDP: Gross Fixed Investment (SWDA, 2020=100)",-1.0,2025-06-30,-1.3,0.24,0.07,3.2,3.0,2025-11-12 19:21:00
"Germany: GDP: Government Consumption (SWDA, 2020=100)",2.1,2025-06-30,3.0,-0.98,-0.65,-1.3,-0.82,2025-11-12 19:21:00
"Germany: Gross Domestic Product (SWDA, 2020=100)",0.28,2025-09-30,0.30,-0.02,-0.01,0.44,0.90,2025-11-12 19:21:00
"Germany: GDP: Total Exports (SWDA, 2020=100)",-1.5,2025-06-30,-0.77,-0.68,-0.14,0.22,-1.6,2025-11-12 19:21:00


In [14]:
growth_stats_collection.dashboard.plots(show_0=True)

##### Nominal GDP

In [15]:
#  Nominal GDP Growth
hc.get_series("DENNGDPP@GERMANY").plot(
    show_0=True,
    tick_suffix="%",
)  # Nominal GDP Growth

### Prices

In [16]:
prices = {
    "EU HICP": "yryr%(F023H@EUDATA)",  # Eurozone
    # "H-CPI (DESTATSIS)":"DENPCHY@GERMANY ",
    "H-CPI (Bundesbank)": "yryr%(DENPCHBK@GERMANY)",
    "H-CPI Core (Bundesbank)": "yryr%(DENPCHXG@GERMANY) ",
    "Retail Price Index": "yryr%(DENPVS@GERMANY)",
    "Producer Price Index": "yryr%(DEGPPT@GERMANY)",  # Disc
    "Bundesbank Projection": "DEAPH@GERMANY",
}

prices_collection = hc.create_collection(list(prices.values()))
prices_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Germany: RPI of Retail Trade and Motor Vehicle Trade excl VAT (NSA, 2020=100)",1.7,2025-10-31,2.0,-0.26,-0.71,0.16,0.73,2025-11-12 19:21:00
"Euro Area 11-20: As Reported Flash HICP Estimate (NSA, 2015=100)",2.1,2025-10-31,2.2,-0.10,-0.24,-0.05,0.11,2025-11-12 19:21:00
"Germany: Harmonized Consumer Price Index (NSA,2015=100)",2.3,2025-10-31,2.4,-0.09,-0.19,0.13,-0.05,2025-11-12 19:21:00
"Germany: PPI: Total Industry excl Construction --DISC(SA, 2000=100)",4.4,2008-12-31,5.3,-0.91,-1.2,-2.3,1.9,2025-11-12 19:21:00
Germany: Bbk Projections: Harmonised Consumer Price Index(%Chg),1.9,2027-12-31,1.5,0.40,nan,nan,nan,2025-11-12 19:21:00
"Germany: HICP excluding Energy and Food (NSA, 2015=100)",2.8,2025-10-31,2.6,0.24,0.61,-0.30,-0.43,2025-11-12 19:22:00


In [17]:
prices_df = prices_collection.df
prices_df.columns = list(prices.keys())
fig = plot_lines(
    prices_df.iloc[:, :3],
    years_limit=8,
    title="<b>Key Inflation Measures</b>",
    tick_suffix="%",
    source="Bundesbank",
)
fig.add_hline(2, line_width=1, annotation_text="Target")  # , annotation="Target")

In [18]:
prices_collection["DEAPH@GERMANY"].plot(tick_suffix="%")

In [19]:
zew_inf_survey = hc.get_series("DENVEIDE@GERMANY")
zew_inf_survey.plot(show_0=True)

In [20]:
# Idiosyncratic stuff
# weekly_price = {
#     "Commodity Price Index": "P156CP@INTWKLY",
#     "Commodity Price Index Ex Energy": "P156CPX@INTWKLY",
#     "Gasoline Price": "P156GRT@INTWKLY",
#     "Pork Prices": "P156HGV@INTWKLY"
# }

# weekly_price_collection = []
# for k, v in weekly_price.items():
#     weekly_price_collection.append(hc.get_series(v))

# weekly_price_collection = TulipCollection(weekly_price_collection)
# weekly_price_collection.dashboard.table()


### Employment

In [21]:
# Employment metrics
employment = {
    "Unemployment Rate": "DESEUR@GERMANY",  # Canada: Employment Insurance: Claims Established
    "BA-X Job Index": "DESEBAX@GERMANY",  # labor demand in the primary labor market and is based on actual openings for jobs and not on surveys
    "Unemployment in broad Sense": "DESE001B@GERMANY",
    "Job Vacancies": "DESE250U@GERMANY",
    "Labor Force": "DESEULF@GERMANY",
    "Ifo Employment Barometer": "DESVIE@GERMANY",
    "DIHK Hiring Intentions": "DENVHEIN@GERMANY",
}


employment_collection = hc.create_collection(list(employment.values()))
employment_collection["DESEUR@GERMANY"].good_is = -1
employment_collection["DESE001B@GERMANY"].good_is = -1
employment_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Germany: LFS: Labor Force (SA, Mil)",44.3,2025-09-30,44.3,0.03,0.72,0.21,0.21,2025-11-12 19:22:00
"Germany: Unemployment in Broader Sense (SA, Persons)","3,147,000",2025-10-31,"3,146,000","1,000",0.03,"32,000","80,000",2025-11-12 19:22:00
"Germany: DIHK Hiring [Employment] Intentions: Balance (NSA, %)",-13.0,2025-12-31,-9.0,-4.0,-0.57,-4.0,-1.0,2025-11-12 19:22:00
"Germany: BA-X Job Index (SWDA, 2015=100)",97.0,2025-10-31,98.0,-1.0,-0.48,-8.0,-10.0,2025-11-12 19:22:00
"Germany: LFS: Unemployment Rate (SA, %)",3.9,2025-09-30,3.8,0.10,1.2,0.30,0.50,2025-11-12 19:22:00
"Germany: Ifo Employment Barometer (SA, 2015=100)",93.5,2025-10-31,92.5,1.0,0.96,-0.40,-0.20,2025-11-12 19:22:00
"Germany: Job Vacancies [Unsubsidized] (SA, Thous)",612.0,2025-10-31,613.0,-1.0,-0.10,-35.0,-64.0,2025-11-12 19:22:00


In [22]:
employment_collection.dashboard.plots(years_limit=10)

### Output Gap

In [23]:
output_gap = hc.get_series("F134FAAA@AMECO")
output_gap.plot(show_0=True)

### Trade

In [24]:
trade_stats = {
    "Current Account Balance % GDP": "DENBCW@GERMANY",
    "Current Account Balance Goods % GDP": "DENBGW@GERMANY",
    "Current Account Balance Services % GDP": "DENBSW@GERMANY",
}

trade_collection = []
for k, v in trade_stats.items():
    trade_collection.append(hc.get_series(v))

trade_collection = TulipCollection(trade_collection)
trade_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Germany: Current Account Balance - DISC (NSA, Mil. EUR)","59,692",2024-12-31,"53,848","5,844",0.65,813.3,"-16,834",2025-11-12 19:22:00
"Germany: Current Account: Goods: Balance - DISC (NSA, Mil. EUR)","44,866",2024-12-31,"55,769","-10,903",-2.3,"-23,142","-16,998",2025-11-12 19:22:00
"Germany: Current Account: Services: Balance - DISC (NSA, Mil. EUR)","-19,198",2024-12-31,"-28,498","9,300",1.9,"1,168","-6,309",2025-11-12 19:22:00


In [25]:
trade_collection.dashboard.plots(show_0=True, mma=4)

### Surveys

In [26]:
surveys = {
    "ZEW Economic Sentiment": "DENVZCG@GERMANY",  # Canada: Business Barometer Index: 3 Month Outlook
    "Sentix": "DENVSGX@GERMANY",
    "Sentix Current": "DENVSH6@GERMANY",
    "Sentix Future": "DENVSH0@GERMANY",
    "GfK Consumer": "DENVCC@GERMANY",
    "DIHK Current": "DENVHECN@GERMANY",
    "DIHK Future": "DENVHBEN@GERMANY",
    "ZEW EA Current Macroeconomic Conditions": "DENVZCE@GERMANY",  # Business Barometer Index: Canada
    "ZEW EA Macroeconomic Expectations 6m": "DENVZEE@GERMANY",  # Canada: News-Based Policy Uncertainty Index
}

surveys_collection = hc.create_collection(list(surveys.values()))
surveys_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Germany: DIHK Business Expectations: Balance (NSA, %)",-12.0,2025-12-31,-10.0,-2.0,-0.16,-2.0,6.0,2025-11-12 19:22:00
"Germany: DIHK Economic Conditions: Balance (NSA, %)",0.00,2025-12-31,0.00,0.00,0.00,0.00,-1.0,2025-11-12 19:22:00
"Euro Area: ZEW Econ Sentiment, Macroecon Expectations [Next 6 Mos](NSA, % Bal)",25.0,2025-11-30,22.7,2.3,0.17,43.5,12.5,2025-11-12 19:22:00
"Germany: ZEW Economic Sentiment, Current Macroeconomic Conditions (NSA, % Bal)",-78.7,2025-11-30,-80.0,1.3,0.14,2.5,12.7,2025-11-12 19:22:00
"Euro Area: ZEW Economic Sentiment, Current Macroeconomic Conditions (NSA, % Bal)",-27.3,2025-11-30,-31.8,4.5,0.55,23.6,16.5,2025-11-12 19:22:00
"Germany: sentix Economic Index, Current Situation (NSA, %Bal)",-38.2,2025-11-30,-36.5,-1.8,-0.25,0.75,7.0,2025-11-12 19:22:00
"Germany: GfK Consumer Climate (NSA, % Balance)",-24.1,2025-11-30,-22.5,-1.6,-0.70,0.20,-5.7,2025-11-12 19:22:00
"Germany: sentix Economic Expectations/Sentiment Index (NSA, %Bal)",-0.50,2025-11-30,2.8,-3.2,-0.41,15.2,12.2,2025-11-12 19:22:00
"Germany: sentix Overall Economic Index (NSA, %Bal)",-20.4,2025-11-30,-17.9,-2.4,-0.39,7.4,9.4,2025-11-12 19:22:00


In [27]:
surveys_collection.dashboard.plots(years_limit=3, show_0=True)

In [28]:
# DEYFGBZ@GERMANY   [Germany: Federal Budget: Financial Deficit(YTD, Mil.EUR)]

### Credit Creation

In [29]:
trend_ngdp = hc.get_nominal_gdp(ISO_3)

In [30]:
trend_ngdp.ts

1960-03-31      38.37
1960-06-30      38.42
1960-09-30      41.70
1960-12-31      41.50
1961-03-31      42.75
               ...   
2024-09-30    1083.90
2024-12-31    1094.78
2025-03-31    1104.37
2025-06-30    1113.00
2025-09-30    1121.08
Freq: QE-DEC, Name: S134NGDP@G10, Length: 263, dtype: float64

In [31]:
# debt_stats = {
#     "Non-Financial Debt % GDP": "S156zdnp@g10",
#     "Household Debt to Disposable Income": "S156zdhi@g10",
#     "Household Debt % GDP": "S156zdhp@g10"
# }

# debt_collection = []
# for k, v in debt_stats.items():
#     debt_collection.append(hc.get_series(v))

# debt_collection = TulipCollection(debt_collection)
# debt_collection.dashboard.table()


In [32]:
# debt_collection.dashboard.plots()

### Budget

In [33]:
government_finance = {
    "Public Debt": "DENFDGT@GERMANY",  # Canada: Business Barometer Index: 3 Month Outlook
    "Govt Budget Surplus/Deficit": "DEYFGBZ@GERMANY",  # Business Barometer Index: Canada
}

government_finance_collection = []
for k, v in government_finance.items():
    government_finance_collection.append(hc.get_series(v))

government_finance_collection = TulipCollection(government_finance_collection)

government_finance_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Germany: Federal Debt: Federal Securities (EOP, NSA, Mil.EUR)","1,759,843",2025-09-30,"1,743,279","16,564",1.6,"79,676","105,549",2025-11-12 19:22:00
"Germany: Federal Budget: Financial Deficit(YTD, Mil.EUR)","-50,200",2025-09-30,"-45,800","-4,400",-0.24,"-28,800","-17,700",2025-11-12 19:22:00


In [34]:
government_finance_collection.dashboard.plots()

### Asset Pricing

In [35]:
# Asset Piricing

# Yields and Stock